In [ ]:
import pymc as pm
import pandas as pd
import arviz as az

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
def plot_hist(sample, **options):
    """Plot a histogram of APACHE_II.
    
    sample: sequence of values
    """
    sns.histplot(sample, stat='probability', discrete=True,
                 alpha=0.5, **options)
    
def legend(**options):
    """Make a legend only if there are labels."""
    handles, labels = plt.gca().get_legend_handles_labels()
    if len(labels):
        plt.legend(**options)
        
def decorate_goals(ylabel='Probability'):
    """Decorate the axes."""
    plt.xlabel('APACHE II socre')
    plt.ylabel(ylabel)
    plt.title('Distribution of APACHE II')
    legend()
    
def plot_kde(sample, **options):
    """Plot a distribution using KDE.
    
    sample: sequence of values
    """
    sns.kdeplot(sample, cut=0, **options)

def decorate_rate(ylabel='Likelihood'):
    """Decorate the axes."""
    plt.xlabel('Apache per patient')
    plt.ylabel(ylabel)
    plt.title('Distribution of APACHE scoring rate')
    legend()

In [ ]:
df = pd.read_csv('Data/apache_sofa_treatment_63.csv', index_col=0)

In [ ]:
sample_poisson = df['APACHE_II']
observed_ndokrw = df[df['Rozpoznania']=='CHF_nieniedokrwienna']['APACHE_II']
observed_dokrw = df[df['Rozpoznania']=='CHF_niedokrwienna']['APACHE_II']
print(sample_poisson.mean().round(2))

In [ ]:
α = 30
β = 1.8

In [ ]:
with pm.Model() as model1:
    μ = pm.Gamma('μ', α, β)
    apache = pm.Poisson('apache', μ, observed=10)
    idata1 = pm.sample(1000)

In [ ]:
sample_posterior = idata1.posterior['μ'][0]
sample_posterior.values.mean().round(2)

In [ ]:
len(sample_posterior)

In [ ]:
az.plot_posterior(sample_posterior)
decorate_rate()

In [ ]:
with model1:
    post_pred = pm.sample_posterior_predictive(idata1)

In [ ]:
post_pred

In [ ]:
sample_post_pred = post_pred.posterior_predictive['apache']
sample_post_pred.values.mean()

In [ ]:
plot_hist(sample_post_pred[0], label='posterior pred')
decorate_goals()

In [ ]:
observed_ndokrw.values

In [ ]:
observed_dokrw.values

In [ ]:
with pm.Model() as model2:
    μ_A = pm.Gamma('μ_A', α, β)
    μ_B = pm.Gamma('μ_B', α, β)
    apache_A = pm.Poisson('ndokrw_A', μ_A, observed=[7,14,1,9])
    apache_B = pm.Poisson('dokrw_B', μ_B, observed=[12,9,10,18])
    idata2 = pm.sample(1000)

In [ ]:
with model2:
    az.plot_trace(idata2);

In [ ]:
mu_A = idata2.posterior['μ_A']
mu_B = idata2.posterior['μ_B']
mu_A.values.mean(), mu_B.values.mean()

In [ ]:
plot_kde(mu_A[0], label='mu_A posterior')
plot_kde(mu_B[0], label='mu_B posterior')
decorate_rate('Density')

In [ ]:
(mu_B.values > mu_A.values).mean()

In [ ]:
with model2:
    post_pred = pm.sample_posterior_predictive(idata2)

In [ ]:
post_pred

In [ ]:
apache_A = post_pred.posterior_predictive['ndokrw_A']
apache_B = post_pred.posterior_predictive['dokrw_B']

In [ ]:
apache_A[0].values.flatten()

In [ ]:
len(apache_A[0])

In [ ]:
plot_hist(apache_A[0].values.flatten(), label='Ndorkw_A')
plot_hist(apache_B[0].values.flatten(), color='C1', label='Dokrw_B')
decorate_goals('PMF')

In [ ]:
win = (apache_B.values > apache_A.values).mean()
win

In [ ]:
lose = (apache_A.values > apache_B.values).mean()
lose

In [ ]:
tie = (apache_A.values == apache_B.values).mean()
tie

In [ ]:
t_A = pm.Exponential.dist(μ_A)
t_A = pm.draw(t_A, 1000)
t_A.mean()

In [ ]:
t_B = pm.Exponential.dist(μ_B)
t_B = pm.draw(t_B, 1000)
t_B.mean()

In [ ]:
prob_win_ot = (t_B < t_A).mean()
prob_win_ot

In [ ]:
# Solution
prob_win = win + tie * prob_win_ot
prob_win

In [ ]:
sample_wins = pm.Binomial.dist(5, prob_win)
sample_wins = pm.draw(sample_wins, 1000)

In [ ]:
# Solution
plot_hist(sample_wins, color='C4')
plt.xlabel('Number of new patients')
plt.ylabel('Probability')
plt.title('Distribution of new patients');

In [ ]:
# Solution

(sample_wins >= 2).mean()